# SCohenLab 2D  Image Processing Workflows notebook (Simplified MCZ)

--------------
# WORKFLOWS OVERVIEW
In order to manage efficient batch processing and UI interaction via Napari we want to leverage `aics-segmentation`s `workflow` tools.  (See also [`napari-allencell-segmenter`](https://github.com/AllenCell/napari-allencell-segmenter)).   This notebook does some initial tests out how to use these tools to build some workflows.  Hopefully these can be eaisily adjusted via a napari plugin.

UPDATE (Dec 2022):  the plugin `organelle-segmenter-plugin' (creates object organelle_segmenter_plugin) was developed largely outside of this notebook.  Hence this notebook has limited expository value.


## Workflow Levels

> BATCH - an "experiments" worth of multi-channel images
> SEGMENTATION WORKFLOWS 
## "Other" organizing principles

Categories
- Extract
- Pre
- Core
- Post 
- Post-post

## future organizing principles
IO
- import: reads file and returns array image
- choose:  i.e. a single Z-slice or ROI -> returns slices, coordinates, indixes
- extract: i.e. select structur channel or create aggregate signal -> returns array image
- export: creates file
- quantify:  returns table of stats





## 2D Workflow example 
- IMAGE
  - 𝟘  _import_ image from .czi file
    - input: file_path
    - output: 4D array, metadata
  - 𝟙  _extract_ optimal Z-slice from 
    - input: 4D array (CH,Z,X,Y)
    - output: 4D array with "chosen" `z_opt` (7,1,nX,nY)
  - 𝟚  binarize STRUCTURE Channel workflows
     - 1️⃣ _SOMA_ workflow
       - input: 4D array (CH,Z,X,Y)  
       - output: SOMA mask (1,1,nX,nY)
     - 2️⃣ _NUCLEI_ workflow
       - input: 4D array (CH,Z,X,Y)  
       - output: NU object(1,1,nX,nY)
     - 3️⃣ _CYTOSOL_ workflow
       - input: SOMA_mask, NU_mask
       - output: CYTO mask (1,1,nX,nY)
     - 4️⃣ _LYSOSOMES_ workflow
        - input: 4D array (CH,Z,X,Y),CYTO_mask
        - output:  LY object
     - 5️⃣ _MITOCHONDRIA_ workflow
        - input: 4D array (CH,z_slice,X,Y),CYTO_mask
        - output:  MT object
     - 6️⃣ _GOLGI complex_ workflow
        - input: 4D array (CH,Z,X,Y),CYTO_mask
        - output:  GL object
     - 7️⃣ _PEROXISOMES_ workflow
        - input: 4D array (CH,Z,X,Y),CYTO_mask
        - output:  PX object
     - 8️⃣ _ENDOPLASMIC RETICULUM_ workflow
        - input: 4D array (CH,Z,X,Y),CYTO_mask
        - output:  ER object
     - 9️⃣ _LIPID BODY_ workflow
        - input: 4D array (CH,Z,X,Y),CYTO_mask
        - output:  LB object 
  - 𝟛  export_ binarized organelle objects
  - 4  quantify binarized organelles





## CHOOSE Z workflow examples
𝟙  _CHOOSE_ optimal Z-slice from 
- inputs: 4D array (CH,Z,X,Y)
 - EXTRACT:
   - create SIG "non-nuclei" signal by adding ch 1-6 
   - create NU signal aggregate
   - return: soma(1,1,nX,nY),
- PRE
    - NU: intensity normalization
    - NU: smoothing
- CORE
  - NU: segmentation
- POST
  - mask total signal to set segmented nuclei pixels to 0
- POST-POST
  - sum over X & Y
  - choose maximum Z
- output:  Z_opt slice



## Structure Channel workflow examples
1️⃣ _SOMA_ workflow
 - inputs: 4D array (CH,Z,X,Y)
 - EXTRACT:
     - create "soma" CH aggregate
     - return: soma(1,1,nX,nY),
  - PRE
    - intensity normalization
    - smoothing
  - CORE
    - segmentation
  - POST
    - filter binarized objects
  - output:  MT object


5️⃣ _MITOCHONDRIA_ workflow
 - inputs: 4D array (CH,Z,X,Y), CYTO_mask
  - EXTRACT:
     - choose mitochondria CH
     - return: (1,1,nX,nY)
  - PRE
    - intensity normalization
    - smoothing
  - CORE
    - segmentation
  - POST
    - filter binarized objects
  - output:  MT object


## ❸. IMAGE PROCESSING ⚙️🩻🔬
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
We will perform computational image analysis on the pictures (in 2D an 3D) to _segment_ the components of interest for measurement.  In other prcoedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, SOMA, CYTOSOL, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing 🌒
- Core-processing (thresholding) 🌕
- Post-processing  🌘

### QC 🚧 WIP 🚧 




## ❹. QUANTIFICATION 📏📐🧮

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Setmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A
# IMPORTS

import  all nescessary packages

we are using `napari` for visualization, and `scipy` `ndimage` and `skimage` for analyzing the image files.  The underlying data format are `numpy` `ndarrays` and tools from  Allen Institute for Cell Science.


In [16]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
from scipy.ndimage import median_filter
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice, edge_preserving_smoothing_3d )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_czi_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray,
                                                                    napari_aiscimageio_reader_fn)
from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles import *

from infer_subc_2d.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROXI_CH ,
                                                                    ER_CH ,
                                                                    LIPID_CH ,
                                                                    RESIDUAL_CH )

%load_ext autoreload
%autoreload 2

test_img_n = TEST_IMG_N

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [3]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [4]:
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [6]:
data_out, meta_out, layer_type = napari_aiscimageio_reader_fn(test_img_name)[0]


In [14]:
meta_out, data_out.shape, layer_type

({'name': ['0 :: None :: Nuclei_Jan22',
   '0 :: None :: Lyso+405_Jan22',
   '0 :: None :: Mito+405_Jan22',
   '0 :: None :: Golgi+405_Jan22',
   '0 :: None :: Peroxy+405_Jan22',
   '0 :: None :: ER+405_Jan22',
   '0 :: None :: BODIPY+405low_Jan22',
   '0 :: None :: Residuals'],
  'channel_axis': 0,
  'scale': (0.5804527163320905, 0.07987165184837318, 0.07987165184837318),
  'metadata': {'aicsimage': <AICSImage [Reader: CziReader, Image-is-in-Memory: False]>,
   'raw_image_metadata': <Element 'ImageDocument' at 0x155ab0720>}},
 (8, 16, 768, 768),
 'image')

In [27]:
test_img_name.split("/")[-1]

'ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi'

In [17]:
viewer = napari.Viewer()
viewer.add_image(
    data_out,
    scale=scale,
    colormap='blue', 
    blending='additive'
)

<Image layer 'data_out' at 0x17beeb5b0>

In [19]:
viewer.add_image(
    data_out,
    channel_axis=meta_out['channel_axis'],
    scale=scale,
    colormap='red', 
    blending='additive'
)

[<Image layer 'Image' at 0x17e8e94c0>,
 <Image layer 'Image [1]' at 0x17ea4cdc0>,
 <Image layer 'Image [2]' at 0x1886c2340>,
 <Image layer 'Image [3]' at 0x18872dcd0>,
 <Image layer 'Image [4]' at 0x1887ea160>,
 <Image layer 'Image [5]' at 0x188858f70>,
 <Image layer 'Image [6]' at 0x1889131c0>,
 <Image layer 'Image [7]' at 0x188984ee0>]

In [6]:
workflow_engine = InferSubC2dWorkflowEngine()

In [7]:
len(workflow_engine.workflow_definitions)

2

In [8]:
wf = workflow_engine.get_executable_workflow_from_config_file(
        "../infer_subc_2d/organelles_config/conf_actb.json",
        img_data[0])


In [9]:

wf.__class__

aicssegmentation.workflow.workflow.Workflow

In [10]:
out = wf.execute_next()

In [11]:
out.__class__

numpy.ndarray

In [12]:
all_functions = workflow_engine._workflow_config.get_all_functions()[0]

In [13]:
wf_definition = wf.workflow_definition